In [1]:
import win32com.client
import pandas as pd
import time
import base64
import subprocess
time.clock = time.time
import boto3
import requests
import os
from datetime import datetime
import pandas as pd

C:\Users\shraDDhar\ds\envs\OCR\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\shraDDhar\ds\envs\OCR\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
C:\Users\shraDDhar\ds\envs\OCR\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
C:\Users\shraDDhar\ds\envs\OCR\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
AWS_ACCESS_KEY_ID=''
AWS_SECRET_ACCESS_KEY=''
aws_access_key_id=AWS_ACCESS_KEY_ID,
aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
region_name='eu-west-1'
# Specify your AWS credentials and region
aws_access_key_id = AWS_ACCESS_KEY_ID
aws_secret_access_key = AWS_SECRET_ACCESS_KEY
# Specify the S3 bucket name and the local file path
bucket_name = 'ocrbiz'
# Initialize Outlook
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
inbox = outlook.GetDefaultFolder(6)  # 6 refers to the inbox folder
# Connect to Amazon Textract
textract_client = boto3.client('textract', region_name='eu-west-1')
session = boto3.Session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name='eu-west-1'
)

In [14]:
def get_emails():
    i=0
    lins={}
    current_date = datetime.now().date()
    fod = current_date.strftime("%d-%m-%Y")
    messages = inbox.Items
    messages.Sort("[ReceivedTime]", True)
    unread_messages = [message for message in messages if message.UnRead == True]
    for message in unread_messages:
        subject = message.Subject
        if "CONVERT_DOC" in subject.upper():
                try:
                    se= message.SenderEmailAddress
                    s=se.split("-")[-1]
                    if s!="A6":
                        print(s)
                    else:
                        s=" "
                except:
                    se= ""
                    s=''
                attachments = message.Attachments
                for attachment in attachments:
                    i+=1
                    if attachment.FileName.endswith('.jpg') or attachment.FileName.endswith('.png'):
                            attachment.SaveAsFile(os.path.join(os.getcwd(), attachment.FileName))
                            imgp = os.path.join(r"C:\Users\shraDDhar\OneDrive - TEVO (Pty) Ltd\Desktop\t\\", attachment.FileName)
                            unid = "CONVERTED_DOC"+str(s)+str(fod)+str(i) # You need to implement this function
                            lins[unid]=[imgp,se]
    return lins

In [18]:
def process_files(lins):
    for key,value in lins.items():
        unid=key
        imgp=value[0]
        local_file_path = imgp.split("\\")[-1]
        # Initialize the S3 client
        s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key, region_name=region_name)
        # Upload the file to S3
        s3.upload_file(local_file_path, bucket_name, local_file_path)
        print("File uploaded successfully to S3 bucket:", bucket_name)
        textract_client = session.client('textract')
        # Specify the S3 bucket and object key for the document you want to analyze
        s3_bucket = 'ocrbiz'
        s3_key = local_file_path
        response = textract_client.detect_document_text(
        Document={
                        'S3Object': {
                            'Bucket': s3_bucket,
                            'Name': s3_key
                        }
                    }
                )
        extracted_text = ''
        for item in response['Blocks']:
            if item['BlockType'] == 'LINE':
                extracted_text += item['Text'] + '\n'
        et=extracted_text.split("\n")
        df=pd.DataFrame(et)
        name=r"t\\"+str(unid)+".xlsx"
        df.to_excel(name)
        outlook = win32com.client.Dispatch("Outlook.Application")
        # Example usage
        subject = "CONVERTED_DOC_REQUEST"
        body = "PLEASE SEE BELOW."
        to = str(value[1])
        attachment_path = name
        mail = outlook.CreateItem(0)
        mail.Subject = subject
        mail.Body = body
        mail.To = to
        if attachment_path:
                attachment = mail.Attachments.Add(attachment_path)
        mail.Send()
        return "1"


In [20]:
k=get_emails()
t=process_files(k)
print(t)

JARED DAVEY
JARED DAVEY
File uploaded successfully to S3 bucket: ocrbiz
1
